In [1]:
from ultralytics import YOLO

path = "/home/jupyter/datasphere/project/model_training/train_with_wandb/runs/detect/train5/weights/"

namePT = "best.pt"

model = YOLO(path + namePT)  
model.export(format="onnx", opset=14) # преобразование в onnx

Ultralytics 8.3.189 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon Processor (Cascadelake))
YOLO11s summary (fused): 100 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs

PyTorch: starting from '/home/jupyter/datasphere/project/model_training/train_with_wandb/runs/detect/train5/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (18.3 MB)

ONNX: starting export with onnx 1.17.0 opset 14...
============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: slimming with onnxslim 0.1.65...
ONNX: export success ✅ 5.6s, saved as '/home/jupyter/datasphere/project/model_training/train_with_wandb/runs/detect/train5/weights/best.onnx' (36.2 MB)

Export complete (6.2s)
Results saved to /home/jupyter/work/resources/model_training/train_with_wandb/runs/detect/train5/weights
Predict:         yolo predict task=det

'/home/jupyter/datasphere/project/model_training/train_with_wandb/runs/detect/train5/weights/best.onnx'

In [2]:
import onnx
from onnx_tf.backend import prepare

nameONNX = 'best.onnx'

onnx_path = path + nameONNX   # путь к ONNX
onnx_model = onnx.load(onnx_path)
tf_rep = prepare(onnx_model, strict=False)
tf_rep.export_graph("yolo_saved_model")      # создание директории saved_model


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to onnx_tf_prefix__model_23_add_1_x, onnx_tf_prefix__model_23_mul_2_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: yolo_saved_model/assets


INFO:tensorflow:Assets written to: yolo_saved_model/assets
INFO:absl:Writing fingerprint to yolo_saved_model/fingerprint.pb


In [3]:
import tensorflow as tf

saved_model_dir = "yolo_saved_model"  
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

# Разрешаем встроенные ops и Select TF Ops (Flex)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS,
  tf.lite.OpsSet.SELECT_TF_OPS
]
converter.allow_custom_ops = True


tflite_model = converter.convert()
open("model.tflite", "wb").write(tflite_model)

37859384

In [10]:
import tensorflow as tf

# Загружаем float модель. Можно из сохранённой модели (SavedModel) или .tflite
# Если у тебя уже .tflite float32, можно конвертировать через from_concrete_functions или from_saved_model

pathToModel="yolo_saved_model"

# Загружаем float модель. Можно из сохранённой модели (SavedModel) или .tflite
# Если у тебя уже .tflite float32, можно конвертировать через from_concrete_functions или from_saved_model

# pathToModel="model.tflite"

converter = tf.lite.TFLiteConverter.from_saved_model(pathToModel)
# Или, если есть keras модель:
# converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]

def representative_dataset():
    for input_value in dataset.take(num_calibration_steps):
        # input_value должен быть numpy массив нужной формы, типа float32
        yield [input_value]

converter.representative_dataset = representative_dataset

# Чтобы гарантировать что всё quantized в int8:
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # или tf.uint8, зависит от модели
converter.inference_output_type = tf.int8

tflite_quant_model = converter.convert()

# Сохраняем
with open('model_int8.tflite', 'wb') as f:
    f.write(tflite_quant_model)



2025-09-20 18:56:03.043627: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-09-20 18:56:03.043693: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-09-20 18:56:03.043996: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: yolo_saved_model
2025-09-20 18:56:03.060057: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-09-20 18:56:03.060108: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: yolo_saved_model
2025-09-20 18:56:03.094768: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-09-20 18:56:03.223076: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: yolo_saved_model
2025-09-20 18:56:03.315913: I tensorflow/cc/saved_model/loader.cc:314] SavedModel load for tags { serve }; Status: success: OK. Took 271917 

ConverterError: <unknown>:0: error: loc(callsite(callsite(fused["SplitV:", "onnx_tf_prefix_/model.2/Split@__inference___call___2271"] at fused["PartitionedCall:", "PartitionedCall@__inference_signature_wrapper_2669"]) at fused["PartitionedCall:", "PartitionedCall"])): 'tf.SplitV' op is neither a custom op nor a flex op
<unknown>:0: note: loc(fused["PartitionedCall:", "PartitionedCall"]): called from
<unknown>:0: note: loc(callsite(callsite(fused["SplitV:", "onnx_tf_prefix_/model.2/Split@__inference___call___2271"] at fused["PartitionedCall:", "PartitionedCall@__inference_signature_wrapper_2669"]) at fused["PartitionedCall:", "PartitionedCall"])): Error code: ERROR_NEEDS_FLEX_OPS
<unknown>:0: error: loc(callsite(callsite(fused["SplitV:", "onnx_tf_prefix_/model.4/Split@__inference___call___2271"] at fused["PartitionedCall:", "PartitionedCall@__inference_signature_wrapper_2669"]) at fused["PartitionedCall:", "PartitionedCall"])): 'tf.SplitV' op is neither a custom op nor a flex op
<unknow